# Exploring_COVID

## Content
- [GENERAL](#GENERAL)
- [MUESTRAS](#MUESTRAS)

## Summary about  each dataset


In [1]:
import pandas as pd
import numpy as np
import os
from sodapy import Socrata

### GENERAL
[Volver](#Exploring_COVID)

**Columns:**
- **'fecha_de_notificaci_n:'** Fecha de notificación de resultado positivo
- **'c_digo_divipola:'** Codigo DIVIPOLA (usado para hacer join luego con el censo)
- **'ciudad_de_ubicaci_n:'** Ciudad de Ubicación
- **'departamento:'** Departamento
- **'atenci_n:'** Tipo de atención
- **'edad:'** Edad de la persona
- **'sexo:'** Sexo
- **'tipo:'** Tipo
- **'estado:'** Estado de salud de la persona
- **'pa_s_de_procedencia:'** País de procedencia
- **'fis:'** Fecha Inicio de Sintomas
- **'fecha_diagnostico:'** Fecha de diagnostico
- **'fecha_recuperado:'** Fecha de recuperación
- **'fecha_reporte_web:'** Fecha de reporte en la Web
- **'tipo_recuperaci_n:'** Tipo de recuperación
- **'codigo_departamento:'** Codigo del departamento
- **'codigo_pais:'** Codigo del pais
- **'pertenencia_etnica:'** Pertenencia etnica
- **'nombre_grupo_etnico:'** Nombre grup etnico
- **'fecha_de_muerte:'** Fecha de muerte

**Preprocessing**
- Set index id_de_caso
- Create new column for "Asintomatico"
- Change dtypes of columns (Dates, ints, strings)
- Get DIVIPOLA codefor department and city.

In [2]:
client = Socrata("www.datos.gov.co", None)
results = client.get("gt2j-8ykr", limit=1000000)
df = pd.DataFrame.from_records(results)

In [3]:
df = df.set_index("id_de_caso", drop=True)

In [4]:
df["Asintomatico"] = (df["fis"]=="Asintomático").astype(int)

In [5]:
df.loc[df["fis"]=="Asintomático", "fis"] = np.nan

In [6]:
df = df.astype({'fecha_de_notificaci_n': np.datetime64,
                'c_digo_divipola': str,
                'ciudad_de_ubicaci_n': str,
                'departamento': str,
                'atenci_n': str,
                'edad': np.int64,
                'sexo': str,
                'tipo': str,
                'estado': str,
                'pa_s_de_procedencia': str,
                'fis': np.datetime64,
                'fecha_diagnostico': np.datetime64,
                'fecha_recuperado': np.datetime64,
                'fecha_reporte_web': np.datetime64,
                'tipo_recuperaci_n': str,
                "codigo_departamento": str,
                "codigo_pais": str,
                "pertenencia_etnica": str,
                'fecha_de_muerte': np.datetime64})

In [7]:
df["divipola_dpto"] = df["c_digo_divipola"].str[:2].astype(int)
df["divipola_mpio"] = df["c_digo_divipola"].str[2:].astype(int)

In [8]:
df.head()

,fecha_de_notificaci_n,c_digo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,pa_s_de_procedencia,...,fecha_reporte_web,tipo_recuperaci_n,codigo_departamento,codigo_pais,pertenencia_etnica,nombre_grupo_etnico,fecha_de_muerte,Asintomatico,divipola_dpto,divipola_mpio
id_de_caso,,,,,,,,,,,,,,,,,,,,,
1,2020-03-02,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,ITALIA,...,2020-03-06,PCR,11,380,Otro,NaN,NaT,0,11,1
2,2020-03-06,76111,Guadalajara de Buga,Valle del Cauca,Recuperado,34,M,Importado,Leve,ESPAÑA,...,2020-03-09,PCR,76,724,Otro,NaN,NaT,0,76,111
3,2020-03-07,05001,Medellín,Antioquia,Recuperado,50,F,Importado,Leve,ESPAÑA,...,2020-03-09,PCR,5,724,Otro,NaN,NaT,0,5,1
4,2020-03-09,05001,Medellín,Antioquia,Recuperado,55,M,Relacionado,Leve,nan,...,2020-03-11,PCR,5,nan,Otro,NaN,NaT,0,5,1
5,2020-03-09,05001,Medellín,Antioquia,Recuperado,25,M,Relacionado,Leve,nan,...,2020-03-11,PCR,5,nan,Otro,NaN,NaT,0,5,1


### MUESTRAS
[Volver](#Exploring_COVID)

**Columns:**


**Preprocessing**
- Set index id_de_caso
- Change dtypes of columns dates and float with nulls.
- Replace , to string

In [11]:
results = client.get("8835-5baf", limit=100000)

# Convert to pandas DataFrame
df_muestras = pd.DataFrame.from_records(results)

In [12]:
df_muestras.columns

Index(['fecha', 'acumuladas', 'amazonas', 'antioquia', 'arauca', 'atlantico',
       'bogota', 'bolivar', 'boyaca', 'caldas', 'caqueta', 'casanare', 'cauca',
       'cesar', 'choco', 'cordoba', 'cundinamarca', 'guainia', 'guajira',
       'guaviare', 'huila', 'magdalena', 'meta', 'narino',
       'norte_de_santander', 'putumayo', 'quindio', 'risaralda', 'san_andres',
       'santander', 'sucre', 'tolima', 'valle_del_cauca', 'vaupes', 'vichada',
       'procedencia_desconocida', 'positivas_acumuladas',
       'negativas_acumuladas', 'positividad_acumulada', 'indeterminadas',
       'barranquilla', 'cartagena', 'santa_marta'],
      dtype='object')

In [13]:
df_muestras = df_muestras[1:]

In [14]:
for column in df_muestras.columns:
    if column=="fecha":
        df_muestras[column] = df_muestras[column].astype(np.datetime64)
    else:
        if column=="positividad_acumulada":
            df_muestras[column] = df_muestras[column].str.replace(",", ".")
        df_muestras[column] = df_muestras[column].astype(np.float64)

In [15]:
df_muestras.shape

(131, 43)

In [16]:
df_muestras

,fecha,acumuladas,amazonas,antioquia,arauca,atlantico,bogota,bolivar,boyaca,caldas,...,vaupes,vichada,procedencia_desconocida,positivas_acumuladas,negativas_acumuladas,positividad_acumulada,indeterminadas,barranquilla,cartagena,santa_marta
1,2020-03-05,636.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-06,739.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-07,810.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-08,813.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-03-09,851.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,2020-07-09,945240.0,11249.0,106711.0,2998.0,26449.0,280355.0,6144.0,15896.0,9798.0,...,1192.0,794.0,8744.0,173208.0,767776.0,18.3,4256.0,82807.0,54225.0,17933.0
128,2020-07-10,968465.0,11299.0,109418.0,3166.0,26935.0,287257.0,6190.0,16138.0,10051.0,...,1192.0,794.0,8744.0,178448.0,785724.0,18.4,4293.0,86142.0,54750.0,18242.0
129,2020-07-11,1006093.0,11597.0,112162.0,3252.0,27715.0,305987.0,6557.0,16263.0,10124.0,...,1192.0,794.0,8748.0,187051.0,814582.0,18.8,4460.0,89673.0,56270.0,19426.0
130,2020-07-12,1031230.0,11605.0,114117.0,3255.0,28435.0,316671.0,6901.0,16439.0,10257.0,...,1478.0,794.0,8749.0,193722.0,832992.0,18.8,4516.0,92732.0,57623.0,19547.0
